# Background subtraction using top-hat in scikit-image and pyclesperanto
This notebook compares different implementations of a background subtraction method.

**Note:** benchmarking results vary heavily depending on image size, kernel size, used operations, parameters and used hardware. Use this notebook to adapt it to your use-case scenario and benchmark on your target hardware. If you have different scenarios or use-cases, you are very welcome to submit your notebook as pull-request!

In [1]:
import pyclesperanto_prototype as cle
import pyclesperanto as pcle

from skimage import morphology
import time

# to measure kernel execution duration properly, we need to set this flag. It will slow down exection of workflows a bit though
cle.set_wait_for_kernel_finish(True)

# selet a GPU with the following in the name. This will fallback to any other GPU if none with this name is found
cle.select_device('RTX')

<NVIDIA GeForce RTX 3050 Ti Laptop GPU on Platform: NVIDIA CUDA (1 refs)>

In [2]:
pcle.set_wait_for_kernel_to_finish(True)
pcle.select_device('RTX')

(OpenCL) NVIDIA GeForce RTX 3050 Ti Laptop GPU (OpenCL 3.0 CUDA)
	Type: GPU
	Compute Units: 20
	Global Memory Size: 4294 MB
	Maximum Object Size: 1073 MB

In [3]:
radius = 10
disk_kernel = morphology.cube(radius)
square_kernel = morphology.ball(radius)

In [4]:
# test data
import numpy as np

test_image = np.random.random([50, 1024, 1024]).astype(np.uint8)

In [5]:
# top-hat (disk) with pyclesperanto_prototype
result_image = None

test_image_gpu = cle.push(test_image)

for i in range(0, 5):
    start_time = time.time()
    result_image = cle.top_hat_sphere(test_image_gpu, result_image, radius_x=radius, radius_y=radius)
    print("pyclesperanto_prototype top-hat-shere duration: " + str(time.time() - start_time))

pyclesperanto_prototype top-hat-shere duration: 0.7882978916168213
pyclesperanto_prototype top-hat-shere duration: 0.6845459938049316
pyclesperanto_prototype top-hat-shere duration: 0.6968636512756348
pyclesperanto_prototype top-hat-shere duration: 0.7028279304504395
pyclesperanto_prototype top-hat-shere duration: 0.7037627696990967


In [6]:
# top-hat (disk) with pyclesperanto
result_image = None

test_image_gpu = pcle.push(test_image)

for i in range(0, 5):
    start_time = time.time()
    result_image = pcle.top_hat_sphere(test_image_gpu, result_image, radius_x=radius, radius_y=radius)
    print("pyclesperanto top-hat-shere duration: " + str(time.time() - start_time))

pyclesperanto top-hat-shere duration: 0.7257959842681885
pyclesperanto top-hat-shere duration: 0.7705111503601074
pyclesperanto top-hat-shere duration: 0.7923500537872314
pyclesperanto top-hat-shere duration: 0.7839434146881104
pyclesperanto top-hat-shere duration: 0.8082761764526367


In [7]:
# top-hat (square) using pyclesperanto_prototype
result_image = None

test_image_gpu = cle.push(test_image)

for i in range(0, 5):
    start_time = time.time()
    result_image = cle.top_hat_box(test_image_gpu, result_image, radius_x=radius, radius_y=radius)
    print("pyclesperanto_prototype top-hat-box duration: " + str(time.time() - start_time))

pyclesperanto_prototype top-hat-box duration: 0.12350010871887207
pyclesperanto_prototype top-hat-box duration: 0.10972404479980469
pyclesperanto_prototype top-hat-box duration: 0.12705731391906738
pyclesperanto_prototype top-hat-box duration: 0.12716078758239746
pyclesperanto_prototype top-hat-box duration: 0.11434435844421387


In [8]:
# top-hat (square) using pyclesperanto
result_image = None

test_image_gpu = pcle.push(test_image)

for i in range(0, 5):
    start_time = time.time()
    result_image = pcle.top_hat_box(test_image_gpu, result_image, radius_x=radius, radius_y=radius)
    print("pyclesperanto top-hat-box duration: " + str(time.time() - start_time))

pyclesperanto top-hat-box duration: 0.07483768463134766
pyclesperanto top-hat-box duration: 0.0700376033782959
pyclesperanto top-hat-box duration: 0.06687235832214355
pyclesperanto top-hat-box duration: 0.06766104698181152
pyclesperanto top-hat-box duration: 0.06505084037780762


In [ ]:
# top-hat (disk) with scikit-image
result_image = None

for i in range(0, 5):
    start_time = time.time()
    result_image = morphology.white_tophat(test_image, footprint=disk_kernel)
    print("skimage top-hat disk duration: " + str(time.time() - start_time))

skimage top-hat disk duration: 10.84842848777771
skimage top-hat disk duration: 10.608829975128174
skimage top-hat disk duration: 10.623921155929565


In [ ]:
# top-hat (square) with scikit-image
result_image = None

for i in range(0, 5):
    start_time = time.time()
    result_image = morphology.white_tophat(test_image, footprint=square_kernel)
    print("skimage top-hat square duration: " + str(time.time() - start_time))